# Imports

Libraries used are tensorflow 2.0, nunmpy, and matplotlib

In [1]:
import numpy as np
import matplotlib as mpl

from matplotlib import pyplot as plt
from tensorflow import keras

# Dataset

In [2]:
from tensorflow.keras.datasets import mnist

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [3]:
print('MNIST Dataset Shape:')
print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(Y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(Y_test.shape))

MNIST Dataset Shape:
X_train: (60000, 28, 28)
Y_train: (60000,)
X_test:  (10000, 28, 28)
Y_test:  (10000,)


# Discriminator

In [4]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

def create_discriminator_model():
    model = Sequential()
    model.add(Conv2D(128, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [5]:
discriminator_model = create_discriminator_model()
discriminator_model.compile(optimizer='adam', loss='binary_crossentropy')

# Generator

In [6]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Conv2DTranspose

dim = 7
depth = 256

def create_generator_model():
    model = Sequential()
    model.add(Dense(dim * dim * depth, activation='relu', input_dim=1))
    model.add(Reshape((dim, dim, depth)))
    model.add(Dropout(0.2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, 3, padding='same'))
    model.add(Activation('relu'))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(1, 3, padding='same'))
    # model.add(Activation('relu'))

    return model

In [7]:
generator_model = create_generator_model()
generator_model.compile(optimizer='adam', loss='binary_crossentropy')

# Training

Define constants

In [8]:
batch_size = 10

Get a random batch from the real numbers

In [9]:
def get_real_batch(size):
    X = X_train[np.random.choice(X_train.shape[0], size, replace=False), :]
    return X

In [ ]:
def get_batch(size, generator):
    # In case the batch size is odd
    generator_batch_size = size // 2
    real_batch_size = size - generator_batch_size

    rand_vals = np.random.rand(generator_batch_size, 100)
    generator_batch = generator.predict(x=rand_vals, batch_size=generator_batch_size)
    real_batch_size = get_real_batch(real_batch_size)

    batch = np.concatenate((generator_batch, real_batch_size))

    return batch, np.concatenate((np.zeros(generator_batch_size), np.ones(real_batch_size)))

(100,)


**Training the discriminator**

Code is taken from https://keras.io/guides/writing_a_training_loop_from_scratch/.

In [ ]:
# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the training dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_train, (-1, 784))
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

